# RAPTOR: Recursive Abstractive Processing for Tree-Organized Retrieval

In [2]:
# NOTE: An OpenAI API key must be set here for application initialization, even if not in use.
# If you're not utilizing OpenAI models, assign a placeholder string (e.g., "not_used").
import os
os.environ["GROQ_API_KEY"] =" "


In [4]:
# # Cinderella story defined in sample.txt
# with open('demo/sample.txt', 'r') as file:
#     text = file.read()

# print(text[:100])

The wife of a rich man fell sick, and as she felt that her end
was drawing near, she called her only


1) **Building**: RAPTOR recursively embeds, clusters, and summarizes chunks of text to construct a tree with varying levels of summarization from the bottom up. You can create a tree from the text in 'sample.txt' using `RA.add_documents(text)`.

2) **Querying**: At inference time, the RAPTOR model retrieves information from this tree, integrating data across lengthy documents at different abstraction levels. You can perform queries on the tree with `RA.answer_question`.

### Building the tree

In [3]:
from raptor import RetrievalAugmentation 
import umap

2025-01-27 12:00:52,271 - Loading faiss with AVX512 support.
2025-01-27 12:00:52,273 - Could not load library with AVX512 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx512'")
2025-01-27 12:00:52,274 - Loading faiss with AVX2 support.
2025-01-27 12:00:52,297 - Successfully loaded faiss with AVX2 support.


In [3]:
# RA = RetrievalAugmentation()

# # construct the tree
# RA.add_documents(text)

NameError: name 'RetrievalAugmentation' is not defined

### Querying from the tree

```python
question = # any question
RA.answer_question(question)
```

In [ ]:
# question = "How did Cinderella reach her happy ending ?"

# answer = RA.answer_question(question=question)

# print("Answer: ", answer)

In [ ]:
# # Save the tree by calling RA.save("path/to/save")
# SAVE_PATH = "demo/cinderella"
# RA.save(SAVE_PATH)

In [ ]:
# # load back the tree by passing it into RetrievalAugmentation

# RA = RetrievalAugmentation(tree=SAVE_PATH)

# answer = RA.answer_question(question=question)
# print("Answer: ", answer)

## Using other Open Source Models for Summarization/QA/Embeddings

If you want to use other models such as Llama or Mistral, you can very easily define your own models and use them with RAPTOR. 

In [5]:
import torch
from raptor import BaseSummarizationModel, BaseQAModel, BaseEmbeddingModel, RetrievalAugmentationConfig
from transformers import AutoTokenizer, pipeline

In [ ]:
# # if you want to use the Gemma, you will need to authenticate with HuggingFace, Skip this step, if you have the model already downloaded
# from huggingface_hub import login
# login()

In [6]:
# ...existing code...
import os
from groq import Groq

class GEMMASummarizationModel(BaseSummarizationModel):
    def __init__(self, model_name="llama-3.3-70b-versatile"):
        self.model_name = model_name
        self.client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

    def summarize(self, context, max_tokens=150):
        messages = [
            {
                "role": "user",
                "content": f"Write a summary of the following, including as many key details as possible: {context}",
            }
        ]
        response = self.client.chat.completions.create(
            model=self.model_name,
            messages=messages,
            max_tokens=max_tokens,
            temperature=0.7,
            top_p=0.95,
        )
        summary = response.choices[0].message.content.strip()
        return summary
# ...existing code...

In [7]:
# ...existing code...
import os
from groq import Groq

class GEMMAQAModel(BaseQAModel):
    def __init__(self, model_name="llama-3.3-70b-versatile"):
        self.model_name = model_name
        self.client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

    def answer_question(self, context, question):
        messages = [
            {
                "role": "user",
                "content": f"Given Context: {context} Give the best full answer to the question {question}",
            }
        ]
        # Remove unsupported arguments (e.g., top_k, top_p)
        response = self.client.chat.completions.create(
            model=self.model_name,
            messages=messages,
            max_tokens=256,
            temperature=0.7,
        )
        return response.choices[0].message.content.strip()
# ...existing code...

In [8]:
from sentence_transformers import SentenceTransformer
class SBertEmbeddingModel(BaseEmbeddingModel):
    def __init__(self, model_name="sentence-transformers/multi-qa-mpnet-base-cos-v1"):
        self.model = SentenceTransformer(model_name)

    def create_embedding(self, text):
        return self.model.encode(text)


In [12]:
RAC = RetrievalAugmentationConfig(summarization_model=GEMMASummarizationModel(), qa_model=GEMMAQAModel(), embedding_model=SBertEmbeddingModel())

2025-01-27 12:01:40,067 - Load pretrained SentenceTransformer: sentence-transformers/multi-qa-mpnet-base-cos-v1
2025-01-27 12:01:40,349 - Use pytorch device: cpu


In [19]:
RA = RetrievalAugmentation(config=RAC)

2025-01-27 12:01:59,891 - Successfully initialized TreeBuilder with Config 
        TreeBuilderConfig:
            Tokenizer: <Encoding 'cl100k_base'>
            Max Tokens: 100
            Num Layers: 5
            Threshold: 0.5
            Top K: 5
            Selection Mode: top_k
            Summarization Length: 100
            Summarization Model: <__main__.GEMMASummarizationModel object at 0x0000021B61650560>
            Embedding Models: {'EMB': <__main__.SBertEmbeddingModel object at 0x0000021B61651730>}
            Cluster Embedding Model: EMB
        
        Reduction Dimension: 10
        Clustering Algorithm: RAPTOR_Clustering
        Clustering Parameters: {}
        
2025-01-27 12:01:59,892 - Successfully initialized ClusterTreeBuilder with Config 
        TreeBuilderConfig:
            Tokenizer: <Encoding 'cl100k_base'>
            Max Tokens: 100
            Num Layers: 5
            Threshold: 0.5
            Top K: 5
            Selection Mode: top_k
            

In [20]:
# ...existing code...
with open('D:/PDF_CONNECT/raptor/csx.txt', 'r', encoding='utf-8', errors='replace') as file:
    text = file.read()

RA.add_documents(text)
# ...existing code...

2025-01-27 12:02:02,544 - Creating Leaf Nodes


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

2025-01-27 12:02:24,929 - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-01-27 12:02:24,931 - Node Texts Length: 368, Summarized Text Length: 100


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-01-27 12:02:25,911 - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-01-27 12:02:25,913 - Node Texts Length: 366, Summarized Text Length: 100


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-01-27 12:02:26,053 - Constructing Layer 1
2025-01-27 12:02:26,053 - Stopping Layer construction: Cannot Create More Layers. Total Layers in tree: 1
2025-01-27 12:02:26,054 - Successfully initialized TreeRetriever with Config 
        TreeRetrieverConfig:
            Tokenizer: <Encoding 'cl100k_base'>
            Threshold: 0.5
            Top K: 5
            Selection Mode: top_k
            Context Embedding Model: EMB
            Embedding Model: <__main__.SBertEmbeddingModel object at 0x0000021B61651730>
            Num Layers: None
            Start Layer: None
        


In [21]:
question = "how to  create Create Material Master – company code level"

answer = RA.answer_question(question=question)

print("Answer: ", answer)

2025-01-27 12:03:16,600 - Using collapsed_tree


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-01-27 12:03:18,257 - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Answer:  To create a Material Master at the company code level, you can follow these steps using the streamlined transaction ZCML:

**Step 1: Access the ZCML Transaction**

* Log in to your SAP system and navigate to the ZCML transaction.
* You can search for the transaction code ZCML in the SAP command prompt or navigate to it through the SAP menu.

**Step 2: Enter Material Master Data**

* On the ZCML initial screen, enter the material number, plant ID, and other required details.
* The plant ID is used to create views such as sales plant, foreign views, purchasing view, MRP views, costing/accounting views, and storage/warehouse views.
* You can also copy defaults from an existing MRP profile or data from an existing material number.

**Step 3: Enter Sales and Purchasing Data**

* Enter the sales organization, distribution channel, and warehouse number (if required).
* Assign a purchasing organization for externally procured materials.

**Step 4: Review and Modify Proposed Values**

